In [52]:
import pandas as pd
import gzip as gz
import sqlite3
from zipfile import ZipFile

In [53]:
imdb = 'data/im.db.zip'
with ZipFile(imdb, 'r') as zip:
    zip.extractall('data/imdb_unzipped')

conn = sqlite3.connect('data/imdb_unzipped/im.db')
        
q = """
SELECT *
FROM sqlite_master
"""
pd.read_sql(q, conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [54]:
q = """
SELECT *
FROM movie_basics
ORDER BY start_year
"""
pd.read_sql(q, conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0146592,Pál Adrienn,Pál Adrienn,2010,136.0,Drama
1,tt0154039,So Much for Justice!,Oda az igazság,2010,100.0,History
2,tt0162942,Children of the Green Dragon,A zöld sárkány gyermekei,2010,89.0,Drama
3,tt0230212,The Final Journey,The Final Journey,2010,120.0,Drama
4,tt0312305,Quantum Quest: A Cassini Space Odyssey,Quantum Quest: A Cassini Space Odyssey,2010,45.0,"Adventure,Animation,Sci-Fi"
...,...,...,...,...,...,...
146139,tt6149054,Fantastic Beasts and Where to Find Them 5,Fantastic Beasts and Where to Find Them 5,2024,NaN,"Adventure,Family,Fantasy"
146140,tt3095356,Avatar 4,Avatar 4,2025,NaN,"Action,Adventure,Fantasy"
146141,tt10300398,Untitled Star Wars Film,Untitled Star Wars Film,2026,NaN,Fantasy
146142,tt5637536,Avatar 5,Avatar 5,2027,NaN,"Action,Adventure,Fantasy"


Inclusion criteria includes movies released over a 10 year period from 2012 to 2021.

In [66]:
q = """
SELECT *
FROM principals
"""
pd.read_sql(q, conn)

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"
...,...,...,...,...,...,...
1028181,tt9692684,1,nm0186469,actor,None,"[""Ebenezer Scrooge""]"
1028182,tt9692684,2,nm4929530,self,None,"[""Herself"",""Regan""]"
1028183,tt9692684,3,nm10441594,director,None,None
1028184,tt9692684,4,nm6009913,writer,writer,None


In [69]:
q = """
SELECT *
FROM persons
"""
pd.read_sql(q, conn)

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,actress
606644,nm9990690,Joo Yeon So,NaN,NaN,actress
606645,nm9991320,Madeline Smith,NaN,NaN,actress
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer


Joined principals and persons to movie_basics but think we need to add and fill movie genres columns first.

In [75]:
q = """
SELECT *
FROM movie_basics
JOIN principals
    USING(movie_id)
JOIN persons
    USING(person_id)
WHERE start_year BETWEEN 2012 AND 2021
ORDER BY start_year 
"""
movie_clean = pd.read_sql(q, conn)

In [76]:
movie_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850657 entries, 0 to 850656
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   movie_id            850657 non-null  object 
 1   primary_title       850657 non-null  object 
 2   original_title      850657 non-null  object 
 3   start_year          850657 non-null  int64  
 4   runtime_minutes     675854 non-null  float64
 5   genres              832573 non-null  object 
 6   Action              0 non-null       object 
 7   ordering            850657 non-null  int64  
 8   person_id           850657 non-null  object 
 9   category            850657 non-null  object 
 10  job                 147321 non-null  object 
 11  characters          325741 non-null  object 
 12  primary_name        850657 non-null  object 
 13  birth_year          187280 non-null  float64
 14  death_year          8500 non-null    float64
 15  primary_profession  805908 non-nul

In [77]:
movie_clean.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,Action,ordering,person_id,category,job,characters,primary_name,birth_year,death_year,primary_profession
0,tt0139613,O Silêncio,O Silêncio,2012,NaN,"Documentary,History",None,1,nm9145395,self,None,"[""Himself - Priest of Vilar de Perdizes""]",António Fontes,NaN,NaN,actor
1,tt0139613,O Silêncio,O Silêncio,2012,NaN,"Documentary,History",None,2,nm10117686,self,None,"[""Himself - Son of a GNR Officer""]",Arlindo Espírito Santo,NaN,NaN,None
2,tt0139613,O Silêncio,O Silêncio,2012,NaN,"Documentary,History",None,3,nm0518037,director,None,None,António Loja Neves,1953.0,2018.0,"actor,director,producer"
3,tt0139613,O Silêncio,O Silêncio,2012,NaN,"Documentary,History",None,4,nm0023406,director,None,None,José Manuel Alves Pereira,NaN,NaN,"editor,director,producer"
4,tt0139613,O Silêncio,O Silêncio,2012,NaN,"Documentary,History",None,5,nm0034378,cinematographer,director of photography,None,Paulo Ares,NaN,NaN,"cinematographer,camera_department,director"


In [58]:
genres = movie_clean['genres'].value_counts().index
genres = list(set(genres))

In [60]:
genres = sorted(genres)


In [61]:
genres_single = []
for x in genres:
    if "," not in x:
        genres_single.append(x)
genres_single

['Action',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western']

In [62]:
movie_clean = """
ALTER TABLE movie_basics
ADD COLUMN Action;
"""
movie_clean = pd.read_sql(movie_clean, conn)

TypeError: 'NoneType' object is not iterable

In [47]:
movie_clean = """
ALTER TABLE movie_basics AS updated_movie_basics
ADD COLUMN Action
ADD COLUMN Adventure 
ADD COLUMN Animation 
ADD COLUMN Biography
ADD COLUMN Comedy
ADD COLUMN Crime
ADD COLUMN Documentary
ADD COLUMN Drama
ADD COLUMN Family
ADD COLUMN Fantasy
ADD COLUMN Game-Show
ADD COLUMN History
ADD COLUMN Horror
ADD COLUMN Music
ADD COLUMN Musical
ADD COLUMN Mystery
ADD COLUMN News
ADD COLUMN Reality-TV
ADD COLUMN Romance
ADD COLUMN Sci-Fi
ADD COLUMN Sport
ADD COLUMN Talk-Show
ADD COLUMN Thriller
ADD COLUMN War
ADD COLUMN Western

"""
movie_clean = pd.read_sql(movie_clean, conn)

DatabaseError: Execution failed on sql '
ALTER TABLE movie_basics AS updated_movie_basics
ADD COLUMN Action
ADD COLUMN Adventure 
ADD COLUMN Animation 
ADD COLUMN Biography
ADD COLUMN Comedy
ADD COLUMN Crime
ADD COLUMN Documentary
ADD COLUMN Drama
ADD COLUMN Family
ADD COLUMN Fantasy
ADD COLUMN Game-Show
ADD COLUMN History
ADD COLUMN Horror
ADD COLUMN Music
ADD COLUMN Musical
ADD COLUMN Mystery
ADD COLUMN News
ADD COLUMN Reality-TV
ADD COLUMN Romance
ADD COLUMN Sci-Fi
ADD COLUMN Sport
ADD COLUMN Talk-Show
ADD COLUMN Thriller
ADD COLUMN War
ADD COLUMN Western

': near "AS": syntax error